In [68]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from pprint import pprint
import json
import time
import requests
from requests.auth import HTTPBasicAuth
import rasterio
from shapely.geometry import GeometryCollection, Polygon, box, shape, mapping

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
os.environ["ACTINIA_USER"] = 'actinia-gdi'
os.environ["ACTINIA_PASSWORD"] = 'actinia-gdi'
os.environ["AUTH"] = 'actinia-gdi:actinia-gdi'
os.environ["ACTINIA_URL"] = 'http://localhost:8088'
os.environ["ACTINIA_URL"] = 'http://localhost:8088'
os.environ["GCS_RESOURCE_BUCKET"] = 'tomorrownow-actinia-dev'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = './env/gcp_tomorrownow_dev_key.json'

ACTINIA_VERSION = 'v3'
ACTINIA_BASEURL = 'http://localhost:8088'
ACTINIA_URL = ACTINIA_BASEURL + "/api/" + ACTINIA_VERSION
ACTINIA_AUTH = HTTPBasicAuth("actinia-gdi", "actinia-gdi")
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
## Set your grass data location
gj.init("../actinia-core-data/grassdb", "nc_spm_08", "PERMANENT")

In [16]:
from datetime import datetime

import pystac
print(pystac.__version__)

from pystac.extensions.projection import ProjectionExtension
from pystac.extensions.view import ViewExtension

1.4.0


# Create a New root STAC catalog

Creating a root catalog to oragnize all data under

In [6]:
savana = pystac.Catalog(id = "savana", description= "STAC collection containing data used by Savana.")

In [7]:
savana.links

[<Link rel=root target=<Catalog id=savana>>]

In [14]:
# Placeholder spatial extent will set from items once they are loaded
sp_extent = pystac.SpatialExtent([None,None,None,None])

## Add child catalog

Adding a child catalog to savana to store land cover land use collections

In [8]:
lclu = pystac.Catalog(id = "lclu", description="A catalog containing land cover land use data collections")
savana.add_child(lclu)

In [9]:
savana.links

[<Link rel=root target=<Catalog id=savana>>,
 <Link rel=child target=<Catalog id=lclu>>]

In [11]:
savana.describe()

* <Catalog id=savana>
    * <Catalog id=lclu>


## Add a collection to the lclu catalog

In [17]:
sp_extent = pystac.SpatialExtent([None,None,None,None])
start_date = datetime.strptime('2001-01-01', '%Y-%m-%d')
end_data = datetime.strptime('2019-01-01', '%Y-%m-%d')
temporal_extent = pystac.TemporalExtent([(start_date, end_data)])
extent = pystac.Extent(sp_extent, temporal_extent)

In [75]:
nlcd = pystac.Collection(id='nlcd', description = 'NLCD datasets 2001 - 2019', extent = extent)
lclu.add_child(nlcd)

In [76]:
savana.describe()

* <Catalog id=savana>
    * <Catalog id=lclu>
        * <Collection id=nlcd>


### Load data from GCS into collectioni

In [20]:
from google.cloud import storage

In [46]:
# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "tomorrownow-actinia-dev"
buckets = storage_client.list_buckets()

In [53]:
blobs = storage_client.list_blobs(bucket_name, prefix="nlcd")
data = [b.name for b in blobs if b.name.endswith('tif')]
print(data)

['nlcd/nlcd_2001_cog.tif', 'nlcd/nlcd_2004_cog.tif', 'nlcd/nlcd_2006_cog.tif', 'nlcd/nlcd_2008_cog.tif', 'nlcd/nlcd_2011_cog.tif', 'nlcd/nlcd_2013_cog.tif', 'nlcd/nlcd_2016_cog.tif', 'nlcd/nlcd_2019_cog.tif']


In [60]:
blobs = storage_client.list_blobs(bucket_name, prefix="nlcd")

for blob in blobs:
    if blob.name.endswith('tif'):
        print("Content-type: {}".format(blob.content_type))
        blob.content_type = "image/tiff; application=geotiff; profile=cloud-optimized"
        blob.patch()
        blob.make_public()

Content-type: image/tiff
Content-type: image/tiff
Content-type: image/tiff
Content-type: image/tiff
Content-type: image/tiff
Content-type: image/tiff
Content-type: image/tiff
Content-type: image/tiff; application=geotiff; profile=cloud-optimized


### Add items to collection

In [77]:
for d in data:
    uri = os.path.join(f'gs://{bucket_name}', d)
    params = {}
    params['id'] = os.path.basename(uri).split('.')[0]
    with rasterio.open(uri) as src:
        params['bbox'] = list(src.bounds)
        params['geometry'] = mapping(box(*params['bbox']))
        
    params['datetime'] = datetime.strptime(f'{d.split("_")[1]}-01-01', '%Y-%m-%d')
    params['properties'] = {}
    i = pystac.Item(**params)
    i.add_asset(key='image', asset=pystac.Asset(href=uri,
                                                title='Geotiff',
                                                media_type=pystac.MediaType.COG))
    nlcd.add_item(i)

In [78]:
# lclu.remove_child('nlcd')
savana.describe()

* <Catalog id=savana>
    * <Catalog id=lclu>
        * <Collection id=nlcd>
          * <Item id=nlcd_2001_cog>
          * <Item id=nlcd_2004_cog>
          * <Item id=nlcd_2006_cog>
          * <Item id=nlcd_2008_cog>
          * <Item id=nlcd_2011_cog>
          * <Item id=nlcd_2013_cog>
          * <Item id=nlcd_2016_cog>
          * <Item id=nlcd_2019_cog>


### Update item extents

In [80]:
nlcd.update_extent_from_items()
nlcd.to_dict()

{'type': 'Collection',
 'id': 'nlcd',
 'stac_version': '1.0.0',
 'description': 'NLCD datasets 2001 - 2019',
 'links': [{'rel': <RelType.ROOT: 'root'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.J

## Normalize Catalogs hrefs

In [81]:
# savana.normalize_hrefs(os.path.join(tmp_dir.name, 'stac'))
savana.normalize_hrefs('./stac_catalog')
savana.save("./savana_stac_catalog")